In [41]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [42]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
import copy
import numpy as np

In [43]:
def load_data():
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    
    # mnist
    train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
    test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
    
    # mnist for each model
    indices = np.random.permutation(len(train_dataset))
    subset1 = Subset(train_dataset, indices[:600])
    subset2 = Subset(train_dataset, indices[600:1200])
    
    train_loader1 = DataLoader(subset1, batch_size=50, shuffle=True)
    train_loader2 = DataLoader(subset2, batch_size=50, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)
    
    return train_loader1, train_loader2, test_loader

In [44]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 128, 3, padding=1)
        self.dropout = nn.Dropout(0.3)
        self.fc1 = nn.Linear(128 * 7 * 7, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = self.dropout(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = torch.relu(self.fc1(x))
        return x

In [45]:
def train_model(model, train_loader, epochs=1):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    
    model.train()
    for epoch in range(epochs):
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            

def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    return 100 * correct / total


In [46]:
def average_model_parameters(models, average_weights):
    averaged_params = copy.deepcopy(models[0].state_dict())
    
    for key in averaged_params.keys():
        averaged_params[key] = sum(weight * models[i].state_dict()[key] for i, weight in enumerate(average_weights))
    
    return averaged_params


In [47]:
def update_model_parameters(model, new_params):
    model.load_state_dict(new_params)

In [48]:
def federated_learning(train_loader1, train_loader2, test_loader):
    model1 = SimpleCNN()
    model2 = SimpleCNN()
    
    rounds = 10
    for i in range(rounds):
        train_model(model1, train_loader1, epochs=5)
        train_model(model2, train_loader2, epochs=5)
        
        acc1_before = evaluate_model(model1, test_loader)
        print(f"Accuracy of model 1 before averaging: {acc1_before:.2f}%")

        acc2_before = evaluate_model(model2, test_loader)
        print(f"Accuracy of model 2 before averaging: {acc2_before:.2f}%")

        average_weights = [0.5, 0.5]
        averaged_params = average_model_parameters([model1, model2], average_weights)
        
        update_model_parameters(model1, averaged_params)
        update_model_parameters(model2, averaged_params)

        

#     acc1_before = evaluate_model(model1, test_loader)
#     print(f"Accuracy of model 1 before averaging: {acc1_before:.2f}%")
    
#     acc2_before = evaluate_model(model2, test_loader)
#     print(f"Accuracy of model 2 before averaging: {acc2_before:.2f}%")

#     average_weights = [0.5, 0.5]
#     averaged_params = average_model_parameters([model1, model2], average_weights)

#     averaged_model = SimpleCNN()
#     update_model_parameters(averaged_model, averaged_params)

    averaged_acc = evaluate_model(averaged_model, test_loader)
    print(f"Accuracy of averaged model: {averaged_acc:.2f}%")

    return acc1_before, acc2_before, averaged_acc


In [49]:
train_loader1, train_loader2, test_loader = load_data()

In [50]:
acc1_before, acc2_before, averaged_acc = federated_learning(train_loader1, train_loader2, test_loader)

Accuracy of model 1 before averaging: 56.02%
Accuracy of model 2 before averaging: 56.52%
Accuracy of model 1 before averaging: 68.51%
Accuracy of model 2 before averaging: 62.86%
Accuracy of model 1 before averaging: 81.10%
Accuracy of model 2 before averaging: 76.12%
Accuracy of model 1 before averaging: 84.79%
Accuracy of model 2 before averaging: 83.91%
Accuracy of model 1 before averaging: 86.63%
Accuracy of model 2 before averaging: 86.30%
Accuracy of model 1 before averaging: 88.04%
Accuracy of model 2 before averaging: 87.51%
Accuracy of model 1 before averaging: 88.08%
Accuracy of model 2 before averaging: 88.94%
Accuracy of model 1 before averaging: 89.56%
Accuracy of model 2 before averaging: 89.57%
Accuracy of model 1 before averaging: 89.34%
Accuracy of model 2 before averaging: 89.79%
Accuracy of model 1 before averaging: 90.30%
Accuracy of model 2 before averaging: 90.32%


NameError: name 'averaged_model' is not defined